# 1. Introduction

<img src="http://pytroll.github.io/logos/pytroll_light_small.png" width="250px"></img>

This notebook is the introduction to the half-day tutorial on the Satpy library.

Satpy is a python library that can be used to work with various earth-observing satellite instrument data and was created by the Pytroll open source software group. Satpy is the result of multiple projects, including Pytroll mpop and CSPP Polar2Grid, combining efforts to build a better project. Development of the Satpy library started in late 2015 and has been designed to assist scientists and researchers in working with these data. One of the main original goals and use cases was to generate high quality, high resolution imagery as quickly as possible in a configured operational server environment. Over the years following its creation, Satpy has grown to support more use cases while also becoming more compatible with other open source python tools. The high level functionality of Satpy includes:

- reading data files
- compositing data products together (ex. RGB images)
- resampling data to new geographic projections
- writing data to on-disk formats
- using the data with other python-based visualization tools

This tutorial is made up of multiple Jupyter notebooks and as a whole will provide an overview of the Satpy library; what it can do and how to use it. Throughout the tutorial we'll use Satpy to read and analyze real satellite instrument data. We'll go over the breadth of features provided by Satpy and leave detailed scientific analysis as an exercise for the reader.

**NOTE**: If you are unable to properly set up a local environment to run the notebooks, be aware that these examples can also be run interactively in the cloud using [Pangeo's BinderHub](http://pangeo-data.org) through a JupyterLab interface. See the [README](https://github.com/pytroll/tutorial-satpy-half-day/blob/main/README.rst) file of this repository for more information (repository linked below).

## Useful links

* [Pytroll Home](http://pytroll.github.io)
* [Pytroll Slack Team](http://pytroll.slack.com)
* [Tutorial Repository](https://github.com/pytroll/tutorial-satpy-half-day)
* [Satpy Repository](https://github.com/pytroll/satpy)
* [Satpy Documentation](https://satpy.readthedocs.io/en/latest/)

# 2. Setup

Before attending an instructor lead version of this tutorial where you will be
executing the notebooks on your local machine (not in the cloud) the necessary
software should be installed and test data downloaded in to the correct
locations. Follow the
[INSTALL](https://github.com/pytroll/tutorial-satpy-half-day/blob/main/INSTALL.md)
instructions for how to create the proper conda environment. Once this notebook
has been started with the proper python environment activated we can run the
following cell to verify the installation. We can run the cell by selecting it
and holding the "Shift" key followed by the "Enter" key.

In the output produced by executing the cell you should see ``ok`` next to each
listed feature that will be used throughout
this tutorial and a return value of ``True``. If not, then your
installation may be broken or corrupt. You may be able to use the error
messages to determine what is missing or broken. Having ``ok`` for the below
features does not guarantee that every part of this tutorial will complete
successfully.

In [ ]:
from satpy.utils import check_satpy
check_satpy(readers=['abi_l1b', 'viirs_sdr'],
            writers=['geotiff', 'cf', 'simple_image'],
            extras=['cartopy', 'geoviews'])

## Download test data

Test data should have been downloaded from the command line before starting the tutorial. If you have not done this already then remove the `#` from the below cell and execute the cell. If you have already done this then please **SKIP** this cell or you may re-download data.

In [ ]:
#%run -e ../download_data.py

# 3. What are we working with?

We will be working with earth-observing meteorological satellite instrument
data. There are many different variations of how the data can be structured,
what it physically represents, how it changes over time, and how it can be used
in a particular type of application. The following is an overview of some common
characteristics of meteorological satellite observations. We'll go in to a few
details as we explore real data later on and completely ignore other
details for simplicity.

## Earth-observing

Satpy operates on data from earth-observing satellite instruments. The data
can be used to study changes in the atmosphere, vegetation, oceans,
pollution, and many others.

<img src="../assets/GOES-16_ABI_RadF_true_color_20180910_170031_GOES-East.small.png" alt="GOES-16 ABI Full Disk True Color" width="400"/>

## Geostationary versus Polar-orbiting

Satellites can be in a high-altitude geostationary orbit or a lower altitude
polar-orbiting orbit. Geostationary satellites typically provide observations
at higher temporal rates of the same region faster (~30s in some), but usually
have a lower spatial resolution and only see part of the Earth.
Polar orbiters are usually at a higher spatial resolution and cover
much more of the Earth, but it takes longer to get all of this coverage.

<table>
    <tr>
        <th style="text-align:center">Geostationary</th>
        <th style="text-align:center">Polar-orbiting</th>
    <tr>
        <td>
            <video width="320" height="240" controls src="../assets/geostationary_orbit.mp4"></video>
        </td>
        <td>
            <video width="320" height="240" controls src="../assets/polar_orbit_with_path.mp4"></video>
        </td>
    </tr>
</table>

<sub>Credit: Clayton Suplinski, SSEC, UW-Madison</sub>

## Imagery

The instrument data we will be working with is imagery data; data from
"imager" instruments. In most cases, these arrays of data points can be
thought of as a 2D image of pixels.


<table>
    <tr>
        <td>
            <img src="../assets/array_print_10.png" alt="Array Values" width="300"/>
        </td>
        <td><font size="30px">&rarr;</font></td>
        <td>
            <img src="../assets/array_imshow_10.png" alt="Array Plot" width="300"/>
        </td>
    </tr>
</table>

## Geolocated

The data we will be looking at is geolocated. We need to be able to assign each
pixel of data to a geographic region. Data footprints that can be somewhat
difficult to describe are typically simplified by specifying
only the center point and the pixel's radius or cell width. We may know the
exact longitude and latitude coordinates of each pixel or we may be given a
gridded version of the data where each pixel is spread uniformly across a
rectangular area.

If you are familiar with projections, we'll get to those later.

<img src="../assets/goes16_abi_cartopy_florida.png" alt="ABI over Florida using Cartopy" width="300"/>

## Bands

The satellite instrument data we will be working with are a collection
of radiation measurements. By taking individual measurements of
the radiation reflected or emitted by objects on Earth, we can get a good
snapshot of the Earth from space. Satellite instruments will typically
have multiple bands or channels where each one measures a specific wavelength
of the electromagnetic spectrum. Each band can show us something different
about the Earth.

<img src="../assets/goes16_abi_4channel_montage.png" alt="ABI C01/C04/C07/C16" width="300"/>

## Sectors

Some geostationary satellites record data for different sectors or regions
of the Earth depending on their purpose and design. For example the GOES-16
ABI instrument has 4 sectors:

- Full Disk (FD or FLDK)
- Continental United States (CONUS)
- Mesoscale 1 (M1)
- Mesoscale 2 (M2)

The below image shows these different sectors and how quickly GOES-16 ABI records data for them. The colors on the image indicate how much of the Earth each data pixel measures.

<img src="../assets/goes16_sectors.png" alt="GOES-16 ABI Sectors" width="300"/>

<sub>Credit: Mat Gunshor, CIMSS</sub>

# 4. New to Notebooks?

If you don't have a lot of experience with Jupyter Notebooks, this section will provide some information to help you through the rest of the tutorial. If you feel comfortable with Notebooks feel free to skip to the next section.

### Cells

Notebooks are made up of individual cells. Cells in this tutorial will be one of two types: Code and Markdown. Code cells are the ones that have real python code that can be executed by you. Markdown cells are specially formatted cells to present instructions or other HTML-like information (images, videos, etc). Markdown cells can not be executed and you shouldn't need to change anything about a Markdown cell.

Code cells can be run by selecting the cell (clicking on it) and hitting `Shift` + `Enter` on your keyboard. This will execute the cell and select the next cell. You can also execute a cell and stay on it by using `Ctrl` + `Enter`. When a code cell is executing it will show `*` in the brackets to the upper-left of the cell. Code cells will execute in the order you run them and do not run in parallel.

### Modes

There are two main modes in Jupyter Notebooks when working with cells: Edit mode and Command mode. In Edit mode you can change the contents of a cell by typing in the cell. You can enter Edit mode by clicking inside a code cell or hitting "Enter" while in Command mode. There will be a green border around a cell when in Edit mode.

In Command mode you can perform operations on the cell like deleting the cell, creating a new cell above or below the current cell, etc. In Command mode cells will have a blue border. You can enter Command mode from Edit mode by hitting "Esc".

### Keyboard Shortcuts

In command mode you can use various keyboard shortcuts:

- `h`: Show help and other keyboard shortcuts
- `a`: Create new cell above current cell
- `b`: Create new cell below current cell
- `dd`: Delete current cell

See the keyboard shortcuts window by typing `h` for more information.

# 5. Next Lesson

It is recommended that as you go through this tutorial you keep the main "file browser" page open to easily select the next notebook. If you've closed this page already you can get to it by clicking:

[**Lesson File Browser**](/tree/notebooks)


It is also recommended that when you are done with a lesson you "Shutdown" that lesson's notebook kernel by selecting the checkbox next to its name on the file browser page and clicking "Shutdown" at the top of the file list. This will end your session for that lesson and remove any progress in the notebook so far. We do this when going through the lessons to reduce the amount of work our system is doing.